2.
Классификатор C4.5 и три его модификации: с оптимизацией гиперпараметра m, гиперпараметра cf и с одновременной оптимизацией обоих гиперпараметров. Эти четыре классификатора сравнивались на 14 наборах данных. На каждом датасете был посчитан AUC каждого классификатора. Данные записаны в файле:

Используя критерий знаковых рангов, проведите попарное сравнение каждого классификатора с каждым. Выберите два классификатора, различие между которыми наиболее статистически значимо.

In [6]:
from itertools import combinations
import pandas as pd
from scipy.stats import wilcoxon

In [58]:
data = pd.read_csv('AUCs.txt', sep = '\t')
data = data.set_index('Unnamed: 0')
data

,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
Unnamed: 0,,,,
adult (sample),0.763,0.768,0.771,0.798
breast cancer,0.599,0.591,0.590,0.569
breast cancer wisconsin,0.954,0.971,0.968,0.967
cmc,0.628,0.661,0.654,0.657
ionosphere,0.882,0.888,0.886,0.898
iris,0.936,0.931,0.916,0.931
liver disorders,0.661,0.668,0.609,0.685
lung cancer,0.583,0.583,0.563,0.625
lymphography,0.775,0.838,0.866,0.875


In [70]:
new_data = []
for x1, x2 in list(combinations(data.columns, 2)):
    stat, p_val = wilcoxon(data[x1].tolist(), data[x2].tolist())
    new_data.append([x1, x2, stat, p_val])

new_data = pd.DataFrame(new_data, columns = ['x1', 'x2', 'stats', 'p_val'])
new_data

/home/andrey/anaconda3/lib/python3.8/site-packages/scipy/stats/morestats.py:2957: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


,x1,x2,stats,p_val
0,C4.5,C4.5+m,6.5,0.010757
1,C4.5,C4.5+cf,43.0,0.861262
2,C4.5,C4.5+m+cf,11.0,0.015906
3,C4.5+m,C4.5+cf,17.0,0.046333
4,C4.5+m,C4.5+m+cf,22.0,0.327826
5,C4.5+cf,C4.5+m+cf,10.0,0.022909


answer = C4.5 и С4.5+m

3.
Сколько статистически значимых на уровне 0.05 различий мы обнаружили?

In [79]:
len(new_data[new_data['p_val'] < 0.05])

4

answer = 4

4.
Сравнивая 4 классификатора между собой, мы проверили 6 гипотез. Давайте сделаем поправку на множественную проверку. Начнём с метода Холма. Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки этим методом?

In [72]:
from statsmodels.sandbox.stats.multicomp import multipletests 

In [73]:
reject, p_corrected, a1, a2 = multipletests(new_data.p_val, 
                                            alpha = 0.05, 
                                            method = 'holm')

In [74]:
new_data['p_corrected'] = p_corrected
new_data['reject'] = reject

In [75]:
new_data

,x1,x2,stats,p_val,p_corrected,reject
0,C4.5,C4.5+m,6.5,0.010757,0.064543,False
1,C4.5,C4.5+cf,43.0,0.861262,0.861262,False
2,C4.5,C4.5+m+cf,11.0,0.015906,0.079532,False
3,C4.5+m,C4.5+cf,17.0,0.046333,0.138998,False
4,C4.5+m,C4.5+m+cf,22.0,0.327826,0.655651,False
5,C4.5+cf,C4.5+m+cf,10.0,0.022909,0.091636,False


answer = 0

6.
Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки методом Бенджамини-Хохберга?

In [76]:
reject, p_corrected, a1, a2 = multipletests(new_data.p_val, 
                                            alpha = 0.05, 
                                            method = 'fdr_bh')

In [78]:
new_data['p_corrected'] = p_corrected
new_data['reject'] = reject
new_data

,x1,x2,stats,p_val,p_corrected,reject
0,C4.5,C4.5+m,6.5,0.010757,0.045818,True
1,C4.5,C4.5+cf,43.0,0.861262,0.861262,False
2,C4.5,C4.5+m+cf,11.0,0.015906,0.045818,True
3,C4.5+m,C4.5+cf,17.0,0.046333,0.069499,False
4,C4.5+m,C4.5+m+cf,22.0,0.327826,0.393391,False
5,C4.5+cf,C4.5+m+cf,10.0,0.022909,0.045818,True


answer = 3